In [1]:
# --------------------------------------------
# 1. Instalar librerías necesarias (solo la primera vez)
# --------------------------------------------
!pip install pystac-client planetary-computer requests shapely folium -q

# --------------------------------------------
# 2. Importar librerías
# --------------------------------------------
from pystac_client import Client
from shapely.geometry import box
from datetime import datetime
import folium
import requests

# --------------------------------------------
# 3. Definir el área de interés y mostrarla en un mapa
# --------------------------------------------
# Bounding box para Caldas (Colombia) – aprox: Manizales y alrededores
bbox = [-75.75, 4.8, -75.35, 5.4]

# Mostrar área en el mapa
m = folium.Map(location=[(bbox[1]+bbox[3])/2, (bbox[0]+bbox[2])/2], zoom_start=11)
folium.Rectangle(bounds=[[bbox[1], bbox[0]], [bbox[3], bbox[2]]], color='blue').add_to(m)
m


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
# --------------------------------------------
# 4. Conectar al STAC API público de Earth Search (puedes cambiarlo por uno privado)
# --------------------------------------------
stac_url = "https://earth-search.aws.element84.com/v1"  # Ejemplo público
client = Client.open(stac_url)

In [ ]:
# --------------------------------------------
# 5. Buscar imágenes Sentinel-2 (L2A) con menos de 20% nubes
# --------------------------------------------
search = client.search(
    collections=["sentinel-2-l2a"],
    bbox=bbox,
    datetime="2024-01-01/2024-01-31",  # Cambia el rango de fechas según tu análisis
    query={"eo:cloud_cover": {"lt": 20}},
    limit=5
)

items = list(search.get_items())
print(f"Imágenes encontradas: {len(items)}")

In [ ]:
# --------------------------------------------
# 6. Mostrar información de las imágenes encontradas
# --------------------------------------------
for i, item in enumerate(items):
    print(f"--- Imagen {i+1} ---")
    print("ID:", item.id)
    print("Fecha:", item.datetime)
    print("Cobertura de nubes:", item.properties["eo:cloud_cover"])
    print("Bandas disponibles:", list(item.assets.keys()))
    print("URL B04 (Red):", item.assets["B04"].href)
    print()

In [ ]:
# --------------------------------------------
# 7. Descargar una banda (ej. B04 - rojo) como GeoTIFF
# --------------------------------------------
# Selecciona una imagen
selected_item = items[0]

# Enlace a la banda roja (B04)
b04_url = selected_item.assets["B04"].href

# Descargar el archivo
response = requests.get(b04_url)

# Guardar en disco
with open("sentinel2_B04.tif", "wb") as f:
    f.write(response.content)

print("Descarga completada: sentinel2_B04.tif")